In [1]:
# generate one path for n assets
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt 

from simulation import *

In [12]:
import torch
import torch.nn as nn

class SimpleLSTMModel(nn.Module):
    def __init__(self, hidden_size, output_size=1):  # Output size is set to 1 by default
        super(SimpleLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        # LSTM layer expects input of shape (batch_size, seq_length, features)
        # For a univariate sequence, features=1
        self.lstm = nn.LSTM(1, hidden_size, batch_first=True)
        # Fully connected layer to map the hidden state output to the desired output size
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Add a batch dimension and feature dimension to x
        # Reshaping x from (n,) to (1, n, 1) to fit LSTM input requirements
        x = x.unsqueeze(0).unsqueeze(-1)  # Now x is of shape [1, seq_length, 1]
        # Process x through the LSTM
        lstm_out, _ = self.lstm(x)
        # Only use the output from the last time step
        # This assumes you're interested in the final output for sequence processing
        out = self.fc(lstm_out[:, -1, :])
        # Squeeze the output to remove 1-dimensions, aiming for a scalar output
        return out.squeeze()

# Initialize the model
hidden_size = 10  # Number of LSTM units in the hidden layer

model = SimpleLSTMModel(hidden_size)
x = torch.randn(50)
f = Functional(model)

In [13]:
dt = 0.01
print(f.partial_t(x, dt))
print(f.partial_x(x, dt))
print(f.partial_xx(x, dt))

0.8193344
-0.009222329
-0.007748604


In [14]:
path = generate_asset_path(mu = np.array([0.1, 0.2]), sigma = np.array([[0.3, 0.4], [0.1, 0.2]]), T = 1, dt = 0.01)   

In [16]:
# define a class of trading policy 
# the trading policy object

naive_states = np.array([[0.1, 0.2], [0.2, 0.3], [0.3, 0.4], [0.4, 0.5], [0.5, 0.6]])
policy = Trading_Policy(f, naive_states)
policy.entropy(path[0], mu = np.array([0.1, 0.2]), sigma = np.array([[0.3, 0.4], [0.1, 0.2]]), dt = 0.01, h = 0.02, gamma = 0.3)

0.4828296572795478

In [17]:
holding_history, wealth_history = simulated_trading(policy, path, 1, np.array([0.1, 0.2]), np.array([[0.3, 0.4], [0.1, 0.2]]), 0.01, 0.1, 0.1) 

In [19]:
wealth_history

array([1.        , 1.00992246, 1.01209849, 1.00868215, 0.95768524,
       0.95718237, 0.96225932, 0.96384248, 0.96155684, 0.9677711 ,
       0.97340999, 0.97798528, 1.04127716, 1.02015273, 0.99079916,
       0.99188268, 0.95329557, 0.91753839, 0.91561831, 0.95066481,
       0.93608202, 0.94575117, 0.90857438, 0.90499673, 0.89752037,
       0.90697818, 0.91930329, 0.94504633, 0.94009147, 0.92146011,
       0.91492661, 0.89589574, 0.93420545, 0.955841  , 1.00505763,
       1.03696045, 1.02823387, 0.94345624, 0.91967819, 0.88093603,
       0.91246976, 0.91679151, 0.88983269, 0.89768684, 0.84912731,
       0.85264281, 0.85989143, 0.87626904, 0.8782967 , 0.86830756,
       0.89859034, 0.90871506, 0.96304566, 0.95561129, 0.94038411,
       0.94305728, 0.94241597, 0.92219178, 0.92253946, 0.94700059,
       0.9818742 , 0.9417577 , 0.96700662, 0.96731278, 0.98719572,
       1.01871292, 1.00464438, 1.01448966, 1.01393933, 1.07669632,
       1.0629594 , 1.08228684, 1.04371267, 1.11424326, 1.11702

In [26]:
# implement the loss function
#torch.autograd.set_detect_anomaly(True)

gamma = 0.3
mu = np.array([0.1, 0.2])
sigma = np.array([[0.3, 0.4], [0.1, 0.2]])


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_value_net = SimpleLSTMModel(24)
#optimizer = torch.optim.Adam(new_value_net.parameters(), lr=0.01)
#optimizer.zero_grad()
wealth_hisotry = torch.tensor(wealth_history, dtype = torch.float32)

# only one path loss
loss = one_path_loss(new_value_net, policy, wealth_history, 0.01, 0.02, 0.3, np.array([0.1, 0.2]), np.array([[0.3, 0.4], [0.1, 0.2]]))

#loss = loss.to(device)
#loss.backward()
#optimizer.step()
print(loss)


KeyboardInterrupt: 

In [ ]:

new_f = Functional(new_value_net)
new_policy = Trading_Policy(new_f, naive_states)
